In [1]:
# GRU 10 FOLD CROSS VALIDATION ON UNIQUE DATASET (based on model 'gru_8_unique'):
# copyright (c) ABDUL HASIB UDDIN <abdulhasibuddin@gmail.com>
# LICENSE: GNU General Public License v3.0
# Courtesy: https://github.com/mchablani/deep-learning/blob/master/sentiment-rnn/Sentiment_RNN.ipynb

In [2]:
import numpy as np
import tensorflow as tf
from timeit import default_timer as timer
from collections import Counter
from string import punctuation
#from google.colab import files

In [3]:
gru_size = 512
gru_layers = 5
k = 10
batch_size = 5
learning_rate = 0.0001
epochs = 3

In [4]:
fileName = "gru_10_fold_cross_validation_12"
checkpointName = "checkpoints/"+fileName+".ckpt"
print(checkpointName)
print(type(checkpointName))

checkpoints/gru_10_fold_cross_validation_12.ckpt
<class 'str'>


In [5]:
#files.upload()
#files.upload()

with open('data_all_unique_dnd_stratified_text.txt', 'r', encoding="utf8") as f:
    tweets = f.read()
with open('data_all_unique_dnd_stratified_labels.txt', 'r', encoding="utf8") as f:
    labels_org = f.read()
    
print('File upload done!')

File upload done!


In [6]:
# Data preprocessing::
#all_text = ''.join([c for c in tweets if c not in punctuation])
all_text = ''.join([c for c in tweets])
tweets = all_text.split('\n')

all_text = ' '.join(tweets)
words = all_text.split()

In [7]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

tweets_ints = []
for each in tweets:
    tweets_ints.append([vocab_to_int[word] for word in each.split()])

In [8]:
# Encoding the labels::
list_labels = []

for l in labels_org.split():
    if l == "depressive":
        list_labels.append(1)
    else:
        list_labels.append(0)
        
labels = np.array(list_labels)
print(len(labels))

1176


In [9]:
tweets_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweets_lens[0]))
print("Maximum tweets length: {}".format(max(tweets_lens)))

Zero-length tweets: 1
Maximum tweets length: 63


In [10]:
# Filter out that tweets with 0 length
tweets_ints = [r[0:200] for r in tweets_ints if len(r) > 0]

In [11]:
from collections import Counter
tweets_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweets_lens[0]))
print("Maximum tweet length: {}".format(max(tweets_lens)))

Zero-length tweets: 0
Maximum tweet length: 63


In [12]:
seq_len = 200
features = np.zeros((len(tweets_ints), seq_len), dtype=int)
# print(features[:10,:100])
for i, row in enumerate(tweets_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
#features[:10,:100]

In [13]:
#split_frac = 0.8

#split_index = int(split_frac * len(features))

#training_validation_x, test_x = features[:split_index], features[split_index:] 
#training_validation_y, test_y = labels[:split_index], labels[split_index:]

training_validation_x = features
training_validation_y = labels

split_train_val = int(len(features)/k)
#split_index = int(split_frac * len(val_x))

#val_x, test_x = val_x[:split_index], val_x[split_index:]
#val_y, test_y = val_y[:split_index], val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train & Validation data set: {}".format(training_validation_x.shape))
print("Train & Validation label set: {}".format(training_validation_y.shape))

############
'''
dataset_split_index = int(len(final_dataset)*0.1)

training_validation_dataset = final_dataset[dataset_split_index:]
testing_dataset = final_dataset[:dataset_split_index]

training_validation_labelset = final_labelset[dataset_split_index:]
testing_labelset = final_labelset[:dataset_split_index]

print(dataset_split_index)

print('len(training_validation_dataset) =',len(training_validation_dataset))
print('len(training_validation_labelset) =',len(training_validation_labelset))

print('len(testing_dataset) =',len(testing_dataset))
print('len(testing_labelset)',len(testing_labelset))
'''

			Feature Shapes:
Train & Validation data set: (1176, 200)
Train & Validation label set: (1176,)


"\ndataset_split_index = int(len(final_dataset)*0.1)\n\ntraining_validation_dataset = final_dataset[dataset_split_index:]\ntesting_dataset = final_dataset[:dataset_split_index]\n\ntraining_validation_labelset = final_labelset[dataset_split_index:]\ntesting_labelset = final_labelset[:dataset_split_index]\n\nprint(dataset_split_index)\n\nprint('len(training_validation_dataset) =',len(training_validation_dataset))\nprint('len(training_validation_labelset) =',len(training_validation_labelset))\n\nprint('len(testing_dataset) =',len(testing_dataset))\nprint('len(testing_labelset)',len(testing_labelset))\n"

In [14]:
n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")


In [15]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [16]:
def gru_cell():
    # Basic GRU cell
    gru = tf.contrib.rnn.GRUCell(gru_size, reuse=tf.get_variable_scope().reuse)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(gru, output_keep_prob=keep_prob)

with tf.name_scope("RNN_layers"):
    # Stack up multiple GRU layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([gru_cell() for _ in range(gru_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [17]:
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

In [18]:
# Output::

with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    tf.summary.histogram('predictions', predictions)
with tf.name_scope('cost'):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

In [19]:
# Validation accuracy::

with tf.name_scope('validation'):
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
# Batching::

def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [21]:
# Training::
saver = tf.train.Saver()
start = timer()
folds_val_acc = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    
    for fold in range(1,k+1):
        print('Fold -',fold,'out of',k,'::')
        print('--------------')

        training_validation_x = training_validation_x.tolist()
        train_x = training_validation_x[:fold*split_train_val-split_train_val]
        train_x += training_validation_x[fold*split_train_val:]
        val_x = training_validation_x[fold*split_train_val-split_train_val:fold*split_train_val]
        
        training_validation_x = np.array(training_validation_x)
        train_x = np.array(train_x)
        val_x = np.array(val_x)
        
        training_validation_y = training_validation_y.tolist()
        train_y = training_validation_y[:fold*split_train_val-split_train_val]
        train_y += training_validation_y[fold*split_train_val:]
        val_y = training_validation_y[fold*split_train_val-split_train_val:fold*split_train_val]
        
        training_validation_y = np.array(training_validation_y)
        train_y = np.array(train_y)
        val_y = np.array(val_y)
        
        print('Training on',len(train_y),'samples & validating on',len(val_y),'samples with batch size',batch_size,'.')
        iteration = 1
        for e in range(1, epochs+1):
            state = sess.run(initial_state)
            for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
                feed = {inputs_: x,
                        labels_: y[:, None],
                        keep_prob: 0.5,
                        initial_state: state}
                summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)

                train_writer.add_summary(summary, iteration)

                if iteration%5==0:
                    print("Epoch: {}/{}".format(e, epochs),
                          "Iteration: {}".format(iteration),
                          "Train loss: {:.4f}".format(loss))

                if iteration%25==0:
                    val_acc = []
                    val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                    for x, y in get_batches(val_x, val_y, batch_size):
                        feed = {inputs_: x,
                                labels_: y[:, None],
                                keep_prob: 1,
                                initial_state: val_state}
                        summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                        val_acc.append(batch_acc)
                    print("Val acc: {:.4f}".format(np.mean(val_acc)))
                iteration +=1
                test_writer.add_summary(summary, iteration)
                saver.save(sess, checkpointName)
                
            #saver.save(sess, checkpointName)
            # After an epoch is completed:
            val_acc = []
            val_state = sess.run(cell.zero_state(batch_size, tf.float32))
            for x, y in get_batches(val_x, val_y, batch_size):
                feed = {inputs_: x,
                        labels_: y[:, None],
                        keep_prob: 1,
                        initial_state: val_state}
                summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                val_acc.append(batch_acc)
            print("Val acc for epoch {:} = {:.4f}".format(e,np.mean(val_acc)))
            test_writer.add_summary(summary, iteration)
            saver.save(sess, checkpointName)
        
        saver.save(sess, checkpointName)        
        # After a fold is completed:
        val_acc = []
        val_state = sess.run(cell.zero_state(batch_size, tf.float32))
        for x, y in get_batches(val_x, val_y, batch_size):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 1,
                    initial_state: val_state}
            summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
            val_acc.append(batch_acc)
            folds_val_acc.append(batch_acc)
        print("Val acc for fold = {:.4f}".format(np.mean(val_acc)))
        test_writer.add_summary(summary, iteration)
        saver.save(sess, checkpointName)
        
        print('--------------\n')
        
duration = timer() - start
print('Time elasped =',duration,'sec(s)')

Fold - 1 out of 10 ::
--------------
Training on 1059 samples & validating on 117 samples with batch size 5 .
Epoch: 1/3 Iteration: 5 Train loss: 0.2658
Epoch: 1/3 Iteration: 10 Train loss: 0.3008
Epoch: 1/3 Iteration: 15 Train loss: 0.2515
Epoch: 1/3 Iteration: 20 Train loss: 0.3014
Epoch: 1/3 Iteration: 25 Train loss: 0.2259
Val acc: 0.5043
Epoch: 1/3 Iteration: 30 Train loss: 0.2492
Epoch: 1/3 Iteration: 35 Train loss: 0.2869
Epoch: 1/3 Iteration: 40 Train loss: 0.2760
Epoch: 1/3 Iteration: 45 Train loss: 0.2017
Epoch: 1/3 Iteration: 50 Train loss: 0.2214
Val acc: 0.5304
Epoch: 1/3 Iteration: 55 Train loss: 0.2669
Epoch: 1/3 Iteration: 60 Train loss: 0.2125
Epoch: 1/3 Iteration: 65 Train loss: 0.3807
Epoch: 1/3 Iteration: 70 Train loss: 0.2127
Epoch: 1/3 Iteration: 75 Train loss: 0.2373
Val acc: 0.5391
Epoch: 1/3 Iteration: 80 Train loss: 0.2953
Epoch: 1/3 Iteration: 85 Train loss: 0.2588
Epoch: 1/3 Iteration: 90 Train loss: 0.2410
Epoch: 1/3 Iteration: 95 Train loss: 0.2041
Epoch: 

Epoch: 1/3 Iteration: 195 Train loss: 0.2585
Epoch: 1/3 Iteration: 200 Train loss: 0.0768
Val acc: 0.5217
Epoch: 1/3 Iteration: 205 Train loss: 0.1656
Epoch: 1/3 Iteration: 210 Train loss: 0.1047
Val acc for epoch 1 = 0.5304
Epoch: 2/3 Iteration: 215 Train loss: 0.5007
Epoch: 2/3 Iteration: 220 Train loss: 0.3650
Epoch: 2/3 Iteration: 225 Train loss: 0.1264
Val acc: 0.5565
Epoch: 2/3 Iteration: 230 Train loss: 0.1332
Epoch: 2/3 Iteration: 235 Train loss: 0.2654
Epoch: 2/3 Iteration: 240 Train loss: 0.1325
Epoch: 2/3 Iteration: 245 Train loss: 0.3140
Epoch: 2/3 Iteration: 250 Train loss: 0.3256
Val acc: 0.6261
Epoch: 2/3 Iteration: 255 Train loss: 0.2565
Epoch: 2/3 Iteration: 260 Train loss: 0.2474
Epoch: 2/3 Iteration: 265 Train loss: 0.2806
Epoch: 2/3 Iteration: 270 Train loss: 0.1309
Epoch: 2/3 Iteration: 275 Train loss: 0.2184
Val acc: 0.6870
Epoch: 2/3 Iteration: 280 Train loss: 0.0721
Epoch: 2/3 Iteration: 285 Train loss: 0.1709
Epoch: 2/3 Iteration: 290 Train loss: 0.0783
Epoch: 

Epoch: 2/3 Iteration: 390 Train loss: 0.1413
Epoch: 2/3 Iteration: 395 Train loss: 0.0056
Epoch: 2/3 Iteration: 400 Train loss: 0.2762
Val acc: 0.6957
Epoch: 2/3 Iteration: 405 Train loss: 0.0050
Epoch: 2/3 Iteration: 410 Train loss: 0.0053
Epoch: 2/3 Iteration: 415 Train loss: 0.0011
Epoch: 2/3 Iteration: 420 Train loss: 0.0002
Val acc for epoch 2 = 0.6087
Epoch: 3/3 Iteration: 425 Train loss: 0.3875
Val acc: 0.6261
Epoch: 3/3 Iteration: 430 Train loss: 0.2017
Epoch: 3/3 Iteration: 435 Train loss: 0.0010
Epoch: 3/3 Iteration: 440 Train loss: 0.2185
Epoch: 3/3 Iteration: 445 Train loss: 0.0010
Epoch: 3/3 Iteration: 450 Train loss: 0.0338
Val acc: 0.6696
Epoch: 3/3 Iteration: 455 Train loss: 0.3036
Epoch: 3/3 Iteration: 460 Train loss: 0.0014
Epoch: 3/3 Iteration: 465 Train loss: 0.0010
Epoch: 3/3 Iteration: 470 Train loss: 0.0550
Epoch: 3/3 Iteration: 475 Train loss: 0.3425
Val acc: 0.7913
Epoch: 3/3 Iteration: 480 Train loss: 0.0122
Epoch: 3/3 Iteration: 485 Train loss: 0.0028
Epoch: 

Epoch: 3/3 Iteration: 585 Train loss: 0.0468
Epoch: 3/3 Iteration: 590 Train loss: 0.0240
Epoch: 3/3 Iteration: 595 Train loss: 0.0012
Epoch: 3/3 Iteration: 600 Train loss: 0.0011
Val acc: 0.7304
Epoch: 3/3 Iteration: 605 Train loss: 0.0148
Epoch: 3/3 Iteration: 610 Train loss: 0.0012
Epoch: 3/3 Iteration: 615 Train loss: 0.0014
Epoch: 3/3 Iteration: 620 Train loss: 0.0000
Epoch: 3/3 Iteration: 625 Train loss: 0.0049
Val acc: 0.7478
Epoch: 3/3 Iteration: 630 Train loss: 0.0019
Val acc for epoch 3 = 0.8087
Val acc for fold = 0.8087
--------------

Fold - 5 out of 10 ::
--------------
Training on 1059 samples & validating on 117 samples with batch size 5 .
Epoch: 1/3 Iteration: 5 Train loss: 0.0002
Epoch: 1/3 Iteration: 10 Train loss: 0.2015
Epoch: 1/3 Iteration: 15 Train loss: 0.1596
Epoch: 1/3 Iteration: 20 Train loss: 0.0794
Epoch: 1/3 Iteration: 25 Train loss: 0.2909
Val acc: 0.9304
Epoch: 1/3 Iteration: 30 Train loss: 0.0020
Epoch: 1/3 Iteration: 35 Train loss: 0.0001
Epoch: 1/3 Ite

Epoch: 1/3 Iteration: 140 Train loss: 0.0002
Epoch: 1/3 Iteration: 145 Train loss: 0.0016
Epoch: 1/3 Iteration: 150 Train loss: 0.0103
Val acc: 0.9652
Epoch: 1/3 Iteration: 155 Train loss: 0.0000
Epoch: 1/3 Iteration: 160 Train loss: 0.1944
Epoch: 1/3 Iteration: 165 Train loss: 0.0888
Epoch: 1/3 Iteration: 170 Train loss: 0.1932
Epoch: 1/3 Iteration: 175 Train loss: 0.0007
Val acc: 0.9391
Epoch: 1/3 Iteration: 180 Train loss: 0.0004
Epoch: 1/3 Iteration: 185 Train loss: 0.0026
Epoch: 1/3 Iteration: 190 Train loss: 0.0001
Epoch: 1/3 Iteration: 195 Train loss: 0.0054
Epoch: 1/3 Iteration: 200 Train loss: 0.0130
Val acc: 0.9565
Epoch: 1/3 Iteration: 205 Train loss: 0.0001
Epoch: 1/3 Iteration: 210 Train loss: 0.0000
Val acc for epoch 1 = 0.9217
Epoch: 2/3 Iteration: 215 Train loss: 0.1994
Epoch: 2/3 Iteration: 220 Train loss: 0.0000
Epoch: 2/3 Iteration: 225 Train loss: 0.0000
Val acc: 0.8783
Epoch: 2/3 Iteration: 230 Train loss: 0.0000
Epoch: 2/3 Iteration: 235 Train loss: 0.0001
Epoch: 

Epoch: 2/3 Iteration: 335 Train loss: 0.0000
Epoch: 2/3 Iteration: 340 Train loss: 0.0000
Epoch: 2/3 Iteration: 345 Train loss: 0.0000
Epoch: 2/3 Iteration: 350 Train loss: 0.0000
Val acc: 0.9478
Epoch: 2/3 Iteration: 355 Train loss: 0.0062
Epoch: 2/3 Iteration: 360 Train loss: 0.0143
Epoch: 2/3 Iteration: 365 Train loss: 0.0007
Epoch: 2/3 Iteration: 370 Train loss: 0.0000
Epoch: 2/3 Iteration: 375 Train loss: 0.0000
Val acc: 0.9391
Epoch: 2/3 Iteration: 380 Train loss: 0.0000
Epoch: 2/3 Iteration: 385 Train loss: 0.0000
Epoch: 2/3 Iteration: 390 Train loss: 0.0004
Epoch: 2/3 Iteration: 395 Train loss: 0.0032
Epoch: 2/3 Iteration: 400 Train loss: 0.0000
Val acc: 0.9565
Epoch: 2/3 Iteration: 405 Train loss: 0.0000
Epoch: 2/3 Iteration: 410 Train loss: 0.0000
Epoch: 2/3 Iteration: 415 Train loss: 0.0000
Epoch: 2/3 Iteration: 420 Train loss: 0.0000
Val acc for epoch 2 = 0.9478
Epoch: 3/3 Iteration: 425 Train loss: 0.3972
Val acc: 0.9478
Epoch: 3/3 Iteration: 430 Train loss: 0.1998
Epoch: 

Epoch: 3/3 Iteration: 530 Train loss: 0.0063
Epoch: 3/3 Iteration: 535 Train loss: 0.1971
Epoch: 3/3 Iteration: 540 Train loss: 0.0000
Epoch: 3/3 Iteration: 545 Train loss: 0.0187
Epoch: 3/3 Iteration: 550 Train loss: 0.0271
Val acc: 0.7043
Epoch: 3/3 Iteration: 555 Train loss: 0.0004
Epoch: 3/3 Iteration: 560 Train loss: 0.0000
Epoch: 3/3 Iteration: 565 Train loss: 0.1990
Epoch: 3/3 Iteration: 570 Train loss: 0.0000
Epoch: 3/3 Iteration: 575 Train loss: 0.0001
Val acc: 0.9217
Epoch: 3/3 Iteration: 580 Train loss: 0.0001
Epoch: 3/3 Iteration: 585 Train loss: 0.0001
Epoch: 3/3 Iteration: 590 Train loss: 0.0001
Epoch: 3/3 Iteration: 595 Train loss: 0.0000
Epoch: 3/3 Iteration: 600 Train loss: 0.0000
Val acc: 0.9826
Epoch: 3/3 Iteration: 605 Train loss: 0.0464
Epoch: 3/3 Iteration: 610 Train loss: 0.0000
Epoch: 3/3 Iteration: 615 Train loss: 0.0000
Epoch: 3/3 Iteration: 620 Train loss: 0.0000
Epoch: 3/3 Iteration: 625 Train loss: 0.0007
Val acc: 0.9652
Epoch: 3/3 Iteration: 630 Train loss

Epoch: 1/3 Iteration: 85 Train loss: 0.0001
Epoch: 1/3 Iteration: 90 Train loss: 0.0003
Epoch: 1/3 Iteration: 95 Train loss: 0.0079
Epoch: 1/3 Iteration: 100 Train loss: 0.0000
Val acc: 0.9913
Epoch: 1/3 Iteration: 105 Train loss: 0.0002
Epoch: 1/3 Iteration: 110 Train loss: 0.0007
Epoch: 1/3 Iteration: 115 Train loss: 0.0001
Epoch: 1/3 Iteration: 120 Train loss: 0.0326
Epoch: 1/3 Iteration: 125 Train loss: 0.0004
Val acc: 0.9739
Epoch: 1/3 Iteration: 130 Train loss: 0.0000
Epoch: 1/3 Iteration: 135 Train loss: 0.0000
Epoch: 1/3 Iteration: 140 Train loss: 0.0001
Epoch: 1/3 Iteration: 145 Train loss: 0.2757
Epoch: 1/3 Iteration: 150 Train loss: 0.0001
Val acc: 0.9652
Epoch: 1/3 Iteration: 155 Train loss: 0.0668
Epoch: 1/3 Iteration: 160 Train loss: 0.0001
Epoch: 1/3 Iteration: 165 Train loss: 0.0019
Epoch: 1/3 Iteration: 170 Train loss: 0.0083
Epoch: 1/3 Iteration: 175 Train loss: 0.0001
Val acc: 0.9913
Epoch: 1/3 Iteration: 180 Train loss: 0.0000
Epoch: 1/3 Iteration: 185 Train loss: 0

In [22]:
# Testing::
'''
test_acc = []
with tf.Session() as sess:
    saver.restore(sess, checkpointName)
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))
'''

'\ntest_acc = []\nwith tf.Session() as sess:\n    saver.restore(sess, checkpointName)\n    test_state = sess.run(cell.zero_state(batch_size, tf.float32))\n    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):\n        feed = {inputs_: x,\n                labels_: y[:, None],\n                keep_prob: 1,\n                initial_state: test_state}\n        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)\n        test_acc.append(batch_acc)\n    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))\n'

In [24]:
print("Accuracy: {:.3f}".format(np.mean(folds_val_acc)))

Accuracy: 0.791
